In [6]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import jReversion as jR
from LDOI import BooleanDOI_processing as BDOIp
from LDOI import qm
import pandas as pd
import numpy as np
import networkx as nx
import concurrent.futures
import par_helper as ph
import random
import robustness as rb

In [3]:
networkModel = 'mapk'

Prefix, Suffix = 'n', 'n'
TEMP = jR.cellcollective(networkModel, Prefix, Suffix, directory='../')

BooleanRuleFileName = TEMP['BooleanRule_filename']
network_name = TEMP['network_name']

NumInputs = TEMP['num_inputs']
NumInputConditions = TEMP['num_input_conditions']

InputConditions = TEMP['input_conditions']

OutputNodes = TEMP['output_nodes']
InputNodes = TEMP['input_nodes']

Mapping = TEMP['mapping']
InverseMapping = TEMP['inverse_mapping']
GRead = TEMP['Gread']
ReadNodes = TEMP['read_nodes']

In [5]:
D = GRead.copy()
D.remove_edges_from(nx.selfloop_edges(D))
din = list(d for n, d in D.in_degree())
dout = list(d for n, d in D.out_degree())

In [2]:
def gen_random_configuration_network_canalizing(din, dout, bias=0.5):
    '''

    The code is originated from the read_ndetwork() by Colin Campbell.
    '''
    def clean_states(x):
        #cleans binary representation of node input states
        out=x[2:]                                                               # Strip leading 0b
        return '0'*(len(inf)-len(out))+out                                      # Append leading 0's as needed

    while True:
        g = nx.DiGraph(nx.directed_configuration_model(din, dout))
        
        input_node = [node for node in g.nodes if g.in_degree(node) == 0]
        output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        if len(input_node) == 0:              # at least one input node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.in_edges(random_node)))
            input_node = [node for node in g.nodes if g.in_degree(node) == 0]
            
        if len(output_node) == 0:            # at least one output node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.out_edges(random_node)))
            output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        
        if len(g.subgraph(input_node + output_node).edges) > 0: continue # output nodes are connected to input nodes
                    
        if nx.is_weakly_connected(g): break

    for n in g.nodes:
        inf = list(g.predecessors(n))
        if len(inf) > 0: 
            g.nodes[n]['update_nodes'] = inf.copy()
            g.nodes[n]['update_rules'] = {}

            bool_states = map(bin,range(2**len(inf)))
            bool_states = map(clean_states,bool_states)
            canalizing_variable = random.randrange(len(inf))
            canalizing_value = int(random.random() < 0.5)
            canalized_value = int(random.random() < 0.5)
            for j in bool_states:
                if j[canalizing_variable] == str(canalizing_value):
                    g.nodes[n]['update_rules'][j] = canalized_value
                else: 
                    g.nodes[n]['update_rules'][j] = int(random.random() < bias)    # Store outcome for every possible input
        else:
#             g.add_edge(n, n)
            g.nodes[n]['update_nodes'] = [n]
            g.nodes[n]['update_rules'] = {'0': 0, '1': 1}

    return g,list(g.nodes)

In [3]:
def r3_random_configuration_network_canalizing(gread, bias=0.5, prefix='n', suffix='n', num_worker=5, sampling_num=10):
    gcopy = gread.copy()
    gcopy.remove_edges_from(nx.selfloop_edges(gcopy))
    din = list(d for n, d in gcopy.in_degree())
    dout = list(d for n, d in gcopy.out_degree())
    g, read_nodes = gen_random_configuration_network_canalizing(din, dout, bias)
    mapping = {}  # nodename to number index
    inverse_mapping = {}  # number index to nodename
    read_nodes_dict = {}
    inverse_read_nodes_dict = {}
    for i, node in enumerate(read_nodes):
        index = prefix + str(i) + suffix
        mapping[str(node)] = index
        inverse_mapping[index] = str(node)
        mapping['~' + str(node)] = '~' + index
        inverse_mapping['~' + index] = '~' + str(node)
        read_nodes_dict[i] = str(node)
        inverse_read_nodes_dict[str(node)] = i
    
    input_nodes = [node for node in g.nodes if g.in_degree(node) == 0]
    output_nodes = [node for node in g.nodes if g.out_degree(node) == 0]
    
    num_inputs = len(input_nodes)
    num_input_conditions = 2 ** num_inputs
    input_conditions = np.ndarray((num_inputs, 2), dtype=object)
    for idx, input_node in enumerate(input_nodes):
        input_conditions[idx, 0] = '~' + str(input_node)
        input_conditions[idx, 1] = str(input_node)
    
    
    output_nodes_ex = []
    output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
    output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
    input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()
    
    GExpanded = ph.par_get_expanded_network(g, prefix=prefix, suffix=suffix, worker=num_worker)
#     GExpanded = BDOIp.Get_expanded_network(g, prefix=prefix, suffix=suffix)
    TempGIOW = jR.get_input_output_relation(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex,
                                            constant_nodes=[])
    LDOIs = TempGIOW['LDOIs']
    GeneLDOIs = TempGIOW['gene_LDOIs']
    Conflicts = TempGIOW['conflicts']
    GeneConflicts = TempGIOW['gene_conflicts']
    IORelation = TempGIOW['io_relation']
    GRemained = TempGIOW['G_remained']

    R0 = jR.identifying_r0_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
    # DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
    R1 = jR.identifying_r1_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
#     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
#     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
    RN = jR.identifying_rn_mutations(GExpanded, mapping, inverse_mapping, input_nodes_ex, output_nodes_ex, input_conditions,
                                     IORelation,
                                     R0['ineffective_mutations'], R1['r1_mutations'])
    
    NodeList = set(read_nodes_dict.values())
    NodeList.difference_update(input_nodes)
    NodeList.difference_update(output_nodes)
    C0, C1, C2, C3 = [], [], [], []
    for NODE in NodeList:
        negNODE = '~' + NODE
        if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
#             nodeClass = 'C0'
            C0.append(NODE)
        elif NODE in RN['rn_mutations']:
            if RN['rn_mutations'][NODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
            else:
#                 nodeClass = 'C2'
                C2.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        elif negNODE in RN['rn_mutations']:
            if RN['rn_mutations'][negNODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        else:
#             nodeClass = 'C3'
            C3.append(NODE)
#             canalizing = IIDC['iid_canalizing'][NODE]
#             unreachable = UN['input_unreachable'][NODE]
    

    robustness_pa = rb.robustness_primary_attractor(g_read=g, state_num=2**sampling_num)
    robustness_ip = rb.robustness_initial_perturbation(g_read=g, state_num=2**sampling_num)
    
    robustness = []
    for key, val in robustness_pa.items():
        robustness.append(val['oe'])
        robustness.append(val['ko'])
#     print(np.mean(robustness))
    
    
    # deterministic_IO
    empty_io = [x for x in IORelation[0,:] if len(x) == 0]
    len_io = [len(x) for x in IORelation[0,:]]
    empty_io_ratio = len(empty_io) / num_input_conditions
    deterministic_io_ratio = sum(len_io) / (num_input_conditions * len(output_nodes))
    
    
    LDOI_union = set()
    for LDOI in LDOIs[0,:]:
        LDOI_union = LDOI_union.union(LDOI)
    LDOI_union = LDOI_union.union([mapping[x] for x in input_conditions.reshape((2*num_inputs,))])
    GSub = nx.subgraph(GExpanded, LDOI_union)
    io_count = 0
    io_pathway_count = 0
    for i in range(num_input_conditions):
        # G_copied = G_expanded.copy()
        BinaryBit = np.binary_repr(i, width=num_inputs)
        InputCondition = [input_conditions[x, int(BinaryBit[x])] for x in range(num_inputs)]
        # source_nodes = input_condition + constant_nodes
        Sources = set([x for x in InputCondition])
        # for node in source:
        #     G_copied.remove_edges_from(list(G_copied.in_edges(node)))
        #     G_copied.remove_edges_from(list(G_copied.in_edges(BDOId.Negation_in_expanded(node))))

        Targets = set([x for x in IORelation[0, i]])
        if len(Targets) > 0:
            for s in Sources:
                for t in Targets:
                    if nx.has_path(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')]):
                        io_count += 1
                        len_shortest_path = nx.shortest_path_length(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')])
                        for p in nx.all_simple_paths(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')], cutoff=2*len_shortest_path):
                            io_pathway_count += 1

    if io_count == 0:
        redundancy = -1
    else:
        redundancy = io_pathway_count/float(io_count)
    
    return {'size_of_network': len(g), 'num_link': len(g.edges), 'num_input': len(input_nodes), 'num_output': len(output_nodes),
            'C0': len(C0), 'C1': len(C1), 'C2': len(C2), 'C3': len(C3), 
            'robustness_pa': np.mean(robustness), 'robustness_ip': robustness_ip,
            'empty_io_ratio': empty_io_ratio, 'deterministic_io_ratio':deterministic_io_ratio,
            'redundancy': redundancy}

In [15]:
test = r3_random_configuration_network_canalizing(GRead, bias=0.5, prefix='n', suffix='n', num_worker=5, sampling_num=10)

In [16]:
result_pd = pd.DataFrame(columns=['network_idx', 'size_of_network', 'num_link', 'C0', 
                                  'C1', 'C2', 'C3', 'robustness_pa', 'robustness_ip', 
                                  'redundancy'])
for idx in range(0, 20):
    temp = r3_random_configuration_network_canalizing(GRead, bias=0.5, prefix='n', suffix='n', num_worker=5, sampling_num=10)
    temp['network_idx'] = idx
    result_pd = result_pd.append(temp, ignore_index=True)

In [17]:
result_pd

,network_idx,size_of_network,num_link,C0,C1,C2,C3,robustness_pa,robustness_ip,redundancy,deterministic_io_ratio,empty_io_ratio,num_input,num_output
0,0.0,53.0,103.0,46.0,7.0,0.0,0.0,0.745283,0.903118,-1.000,0.000000,1.000,4.0,3.0
1,1.0,53.0,103.0,30.0,16.0,3.0,4.0,0.655660,0.919461,16.000,0.166667,0.500,4.0,3.0
2,2.0,53.0,102.0,36.0,14.0,1.0,2.0,0.764151,0.924528,-1.000,0.000000,1.000,4.0,3.0
3,3.0,53.0,103.0,46.0,5.0,1.0,1.0,0.745283,0.924528,-1.000,0.000000,1.000,4.0,3.0
4,4.0,53.0,104.0,40.0,12.0,1.0,0.0,0.741352,0.875571,-1.000,0.000000,1.000,4.0,3.0
5,5.0,53.0,103.0,45.0,8.0,0.0,0.0,0.721698,0.909161,-1.000,0.000000,1.000,4.0,3.0
6,6.0,53.0,102.0,44.0,8.0,0.0,1.0,0.650943,0.923441,-1.000,0.000000,1.000,4.0,3.0
7,7.0,53.0,103.0,38.0,14.0,0.0,1.0,0.731132,0.898124,-1.000,0.000000,1.000,4.0,3.0
8,8.0,53.0,102.0,46.0,6.0,0.0,1.0,0.613208,0.867925,-1.000,0.000000,1.000,4.0,3.0
9,9.0,53.0,101.0,40.0,9.0,0.0,4.0,0.535040,0.854382,-1.000,0.000000,1.000,4.0,3.0


In [18]:
for idx in range(20, 50):
    temp = r3_random_configuration_network_canalizing(GRead, bias=0.5, prefix='n', suffix='n', num_worker=5, sampling_num=10)
    temp['network_idx'] = idx
    result_pd = result_pd.append(temp, ignore_index=True)

In [19]:
result_pd

,network_idx,size_of_network,num_link,C0,C1,C2,C3,robustness_pa,robustness_ip,redundancy,deterministic_io_ratio,empty_io_ratio,num_input,num_output
0,0.0,53.0,103.0,46.0,7.0,0.0,0.0,0.745283,0.903118,-1.000,0.000000,1.000,4.0,3.0
1,1.0,53.0,103.0,30.0,16.0,3.0,4.0,0.655660,0.919461,16.000,0.166667,0.500,4.0,3.0
2,2.0,53.0,102.0,36.0,14.0,1.0,2.0,0.764151,0.924528,-1.000,0.000000,1.000,4.0,3.0
3,3.0,53.0,103.0,46.0,5.0,1.0,1.0,0.745283,0.924528,-1.000,0.000000,1.000,4.0,3.0
4,4.0,53.0,104.0,40.0,12.0,1.0,0.0,0.741352,0.875571,-1.000,0.000000,1.000,4.0,3.0
5,5.0,53.0,103.0,45.0,8.0,0.0,0.0,0.721698,0.909161,-1.000,0.000000,1.000,4.0,3.0
6,6.0,53.0,102.0,44.0,8.0,0.0,1.0,0.650943,0.923441,-1.000,0.000000,1.000,4.0,3.0
7,7.0,53.0,103.0,38.0,14.0,0.0,1.0,0.731132,0.898124,-1.000,0.000000,1.000,4.0,3.0
8,8.0,53.0,102.0,46.0,6.0,0.0,1.0,0.613208,0.867925,-1.000,0.000000,1.000,4.0,3.0
9,9.0,53.0,101.0,40.0,9.0,0.0,4.0,0.535040,0.854382,-1.000,0.000000,1.000,4.0,3.0


In [20]:
result_pd.to_csv('../data/mapk_canalizing.csv')

In [4]:
networkModel =[ #'bortezomib',
                # 'igvh',
                #'apoptosis',
                # 'aurora',
                #'bt474_long',
                'bt474_short',
                # 'cd4t',
                'colitis',
                'death',
                # 'egfr',
                # 'erbb',
                # 'fa_brca',
                # 'fa_check',
                #'hcc1954_long',
                'hcc1954_short',
                'hgf',
                'mammalian',
                # 'mammalian_2006',
                #'mapk',
                'oxidative',
                # 'pro_inflammatory',
                #'fibroblasts',
                #'skbr3_long',
                'skbr3_short',
                'tlgl_2008',
                'tlgl_2011',
                # 'tlgl_2011_reduced',
                # 'prostate',
                'migration']

In [ ]:
for Model in networkModel:
    Prefix, Suffix = 'n', 'n'
    TEMP = jR.cellcollective(Model, Prefix, Suffix, directory='../')

    BooleanRuleFileName = TEMP['BooleanRule_filename']
    network_name = TEMP['network_name']

    NumInputs = TEMP['num_inputs']
    NumInputConditions = TEMP['num_input_conditions']

    InputConditions = TEMP['input_conditions']

    OutputNodes = TEMP['output_nodes']
    InputNodes = TEMP['input_nodes']
    
    Mapping = TEMP['mapping']
    InverseMapping = TEMP['inverse_mapping']
    GRead = TEMP['Gread']
    ReadNodes = TEMP['read_nodes']
    
    result_pd = pd.DataFrame(columns=['network_idx', 'size_of_network', 'num_link', 'C0', 
                                  'C1', 'C2', 'C3', 'robustness_pa', 'robustness_ip', 
                                  'redundancy'])
    for idx in range(0, 50):
        temp = r3_random_configuration_network_canalizing(GRead, bias=0.5, prefix='n', suffix='n', num_worker=10, sampling_num=10)
        temp['network_idx'] = idx
        result_pd = result_pd.append(temp, ignore_index=True)
        
    result_pd.to_csv('../data/'+ Model + '_canalizing.csv')

In [24]:
Model = 'mapk'
Prefix, Suffix = 'n', 'n'
TEMP = jR.cellcollective(Model, Prefix, Suffix, directory='../')

BooleanRuleFileName = TEMP['BooleanRule_filename']
network_name = TEMP['network_name']

NumInputs = TEMP['num_inputs']
NumInputConditions = TEMP['num_input_conditions']

InputConditions = TEMP['input_conditions']

OutputNodes = TEMP['output_nodes']
InputNodes = TEMP['input_nodes']

Mapping = TEMP['mapping']
InverseMapping = TEMP['inverse_mapping']
GRead = TEMP['Gread']
ReadNodes = TEMP['read_nodes']

result_pd = pd.DataFrame(columns=['network_idx', 'size_of_network', 'num_link', 'C0', 
                              'C1', 'C2', 'C3', 'robustness_pa', 'robustness_ip', 
                              'redundancy'])
for idx in range(0, 500):
    temp = r3_random_configuration_network_canalizing(GRead, bias=0.5, prefix='n', suffix='n', num_worker=10, sampling_num=10)
    temp['network_idx'] = idx
    result_pd = result_pd.append(temp, ignore_index=True)

result_pd.to_csv('../data/'+ Model + '_2_canalizing.csv')

In [9]:
def r3_test(g, read_nodes, model_name, prefix='n', suffix='n', num_worker=10, sampling_num=10):
    gcopy = g.copy()
    gcopy.remove_edges_from(nx.selfloop_edges(gcopy))
    
    if len(gcopy) < sampling_num:
        sampling_num = len(gcopy)
    
    mapping = {}  # nodename to number index
    inverse_mapping = {}  # number index to nodename
    read_nodes_dict = {}
    inverse_read_nodes_dict = {}
    for i, node in enumerate(read_nodes):
        index = prefix + str(i) + suffix
        mapping[str(node)] = index
        inverse_mapping[index] = str(node)
        mapping['~' + str(node)] = '~' + index
        inverse_mapping['~' + index] = '~' + str(node)
        read_nodes_dict[i] = str(node)
        inverse_read_nodes_dict[str(node)] = i

    input_nodes = [node for node in gcopy.nodes if gcopy.in_degree(node) == 0]
    output_nodes = [node for node in gcopy.nodes if gcopy.out_degree(node) == 0]

    num_inputs = len(input_nodes)
    num_input_conditions = 2 ** num_inputs
    input_conditions = np.ndarray((num_inputs, 2), dtype=object)
    for idx, input_node in enumerate(input_nodes):
        input_conditions[idx, 0] = '~' + str(input_node)
        input_conditions[idx, 1] = str(input_node)


    output_nodes_ex = []
    output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
    output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
    input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()
    try:
        GExpanded = nx.read_gml('networks/' + model_name+ '_expanded_network.gml')
    except IOError:
        GExpanded = ph.par_get_expanded_network(gcopy, prefix=prefix, suffix=suffix, worker=num_worker)
        
    #     GExpanded = BDOIp.Get_expanded_network(g, prefix=prefix, suffix=suffix)
    TempGIOW = jR.get_input_output_relation(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex,
                                            constant_nodes=[])
    LDOIs = TempGIOW['LDOIs']
    GeneLDOIs = TempGIOW['gene_LDOIs']
    Conflicts = TempGIOW['conflicts']
    GeneConflicts = TempGIOW['gene_conflicts']
    IORelation = TempGIOW['io_relation']
    GRemained = TempGIOW['G_remained']

    R0 = jR.identifying_r0_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
    # DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
    R1 = jR.identifying_r1_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
    #     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
    #     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
    RN = jR.identifying_rn_mutations(GExpanded, mapping, inverse_mapping, input_nodes_ex, output_nodes_ex, input_conditions,
                                     IORelation,
                                     R0['ineffective_mutations'], R1['r1_mutations'])

    NodeList = set(read_nodes_dict.values())
    NodeList.difference_update(input_nodes)
    NodeList.difference_update(output_nodes)
    C0, C1, C2, C3 = [], [], [], []
    for NODE in NodeList:
        negNODE = '~' + NODE
        if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
    #             nodeClass = 'C0'
            C0.append(NODE)
        elif NODE in RN['rn_mutations']:
            if RN['rn_mutations'][NODE] == 'R1':
    #                 nodeClass = 'C1'
                C1.append(NODE)
    #                 canalizing = IIDC['iid_canalizing'][NODE]
    #                 unreachable = UN['input_unreachable'][NODE]
            else:
    #                 nodeClass = 'C2'
                C2.append(NODE)
    #                 canalizing = IIDC['iid_canalizing'][NODE]
    #                 unreachable = UN['input_unreachable'][NODE]
        elif negNODE in RN['rn_mutations']:
            if RN['rn_mutations'][negNODE] == 'R1':
    #                 nodeClass = 'C1'
                C1.append(NODE)
    #                 canalizing = IIDC['iid_canalizing'][NODE]
    #                 unreachable = UN['input_unreachable'][NODE]
        else:
    #             nodeClass = 'C3'
            C3.append(NODE)
    #             canalizing = IIDC['iid_canalizing'][NODE]
    #             unreachable = UN['input_unreachable'][NODE]


    robustness_pa = rb.robustness_primary_attractor(g_read=g, state_num=2**sampling_num)
    robustness_ip = rb.robustness_initial_perturbation(g_read=g, state_num=2**sampling_num)

    robustness = []
    for key, val in robustness_pa.items():
        robustness.append(val['oe'])
        robustness.append(val['ko'])
    #     print(np.mean(robustness))


    # deterministic_IO
    empty_io = [x for x in IORelation[0,:] if len(x) == 0]
    len_io = [len(x) for x in IORelation[0,:]]
    empty_io_ratio = len(empty_io) / num_input_conditions
    deterministic_io_ratio = sum(len_io) / (num_input_conditions * len(output_nodes))


    LDOI_union = set()
    for LDOI in LDOIs[0,:]:
        LDOI_union = LDOI_union.union(LDOI)
    LDOI_union = LDOI_union.union([mapping[x] for x in input_conditions.reshape((2*num_inputs,))])
#     GSub = nx.subgraph(GExpanded, LDOI_union)
    io_count = 0
    io_pathway_count = 0
    for i in range(num_input_conditions):
        # G_copied = G_expanded.copy()
        BinaryBit = np.binary_repr(i, width=num_inputs)
        InputCondition = [input_conditions[x, int(BinaryBit[x])] for x in range(num_inputs)]
        # source_nodes = input_condition + constant_nodes
        Sources = set([x for x in InputCondition])
        # for node in source:
        #     G_copied.remove_edges_from(list(G_copied.in_edges(node)))
        #     G_copied.remove_edges_from(list(G_copied.in_edges(BDOId.Negation_in_expanded(node))))

        Targets = set([x for x in IORelation[0, i]])
        if len(Targets) > 0:
            for s in Sources:
                for t in Targets:
                    if nx.has_path(G=gcopy, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')]):
                        io_count += 1
                        len_shortest_path = nx.shortest_path_length(G=gcopy, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')])
                        for p in nx.all_simple_paths(G=gcopy, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')], cutoff=2*len_shortest_path):
                            io_pathway_count += 1

    if io_count == 0:
        redundancy = -1
    else:
        redundancy = io_pathway_count/float(io_count)

    return {'size_of_network': len(g), 'num_link': len(g.edges), 'num_input': len(input_nodes), 'num_output': len(output_nodes),
            'C0': len(C0), 'C1': len(C1), 'C2': len(C2), 'C3': len(C3), 
            'robustness_pa': np.mean(robustness), 'robustness_ip': robustness_ip,
            'empty_io_ratio': empty_io_ratio, 'deterministic_io_ratio':deterministic_io_ratio,
            'redundancy': redundancy}

In [ ]:
original_pd = pd.DataFrame(columns=['model', 'size_of_network', 'num_link', 'C0', 
                              'C1', 'C2', 'C3', 'robustness_pa', 'robustness_ip', 
                              'redundancy'])
Model_list = ['bortezomib',
                # 'igvh',
                'apoptosis',
                # 'aurora',
#                 'bt474_long',
                'bt474_short',
                # 'cd4t',
                'colitis',
                'death',
                # 'egfr',
                # 'erbb',
                # 'fa_brca',
                # 'fa_check',
#                 'hcc1954_long',
                'hcc1954_short',
                'hgf',
                'mammalian',
                # 'mammalian_2006',
                'mapk',
                'oxidative',
                # 'pro_inflammatory',
                'fibroblasts',
#                 'skbr3_long',
                'skbr3_short',
                'tlgl_2008',
                'tlgl_2011',
                # 'tlgl_2011_reduced',
                # 'prostate',
                'migration']
for Model in Model_list:
    Prefix, Suffix = 'n', 'n'
    TEMP = jR.cellcollective(Model, Prefix, Suffix, directory='../')

    BooleanRuleFileName = TEMP['BooleanRule_filename']
    network_name = TEMP['network_name']

    NumInputs = TEMP['num_inputs']
    NumInputConditions = TEMP['num_input_conditions']

    InputConditions = TEMP['input_conditions']

    OutputNodes = TEMP['output_nodes']
    InputNodes = TEMP['input_nodes']

    Mapping = TEMP['mapping']
    InverseMapping = TEMP['inverse_mapping']
    GRead = TEMP['Gread']
    ReadNodes = TEMP['read_nodes']
    temp = r3_test(GRead, ReadNodes, Model, sampling_num=20)
    temp['model'] = Model
    original_pd = original_pd.append(temp, ignore_index=True)



In [42]:
original_pd.to_csv('../data/original.csv')